# 토치텍스트(TorchText)의 batch_first

## 1. 훈련 데이터와 테스트 데이터로 분리하기

In [1]:
import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7fdc1d3b3a50>)

In [3]:
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [4]:
df.shape

(50000, 2)

In [5]:
train_df = df[:25000]
test_df = df[25000:]

In [6]:
train_df = df.to_csv('train_data.csv', index=False)
test_df = df.to_csv('test_data.csv', index=False)

## 2. 필드 정의하기(torchtext.data)

In [9]:
from torchtext.legacy import data

# batch_first 인자를 True로 설정
TEXT = data.Field(sequential=True, use_vocab=True, lower=True, tokenize=str.split, batch_first=True, fix_length=20)
LABEL = data.Field(sequential=False, use_vocab=False, batch_first=False, is_target=True)

- batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)

## 3. 데이터셋 / 단어 집합 / 데이터로더 만들기

In [12]:
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import Iterator

# TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행합니다.
train_data, test_data = TabularDataset.splits(path='.', train='train_data.csv', test='test_data.csv', format='csv', fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

# 정의한 필드에 build_vocab() 도구를 사용하면 단어 집합을 생성한다.
TEXT.build_vocab(train_data ,min_freq=10, max_size=10000)

In [14]:
batch_size = 5

train_loader = Iterator(dataset=train_data, batch_size=batch_size)
test_loader = Iterator(dataset=test_data, batch_size=batch_size)

batch = next(iter(train_loader))

In [15]:
print(batch.text)

tensor([[ 610,  610,  610,  610,  610,   48,  408,   64,    9, 2974, 3015, 2354,
          202,   26,   71, 1906,    6,    0,  185,   28],
        [ 451,  868,    2,    0,    0,   34,  276,   36,    0, 5154,    2,   25,
          315, 8129,    7,    2,   59, 1600,  991,   49],
        [   9,  202,    9,   14,    3,  107, 9742,   50,    9, 6415,    2,  638,
            6,  109,    0, 2249,  706, 1751, 2294,    6],
        [1926,    6,    2,  204,   10,   20,   14, 1129,    8,    0,   12,    7,
          595,  823,    6,  180,  123,    0,    0,  124],
        [   9,   14,    0,   15,    9,  267,    3,  533,  397, 7374,    5,  379,
            0, 1673,    0,    0,   34,    2,   88, 8212]])


In [16]:
print(batch.text.shape)

torch.Size([5, 20])


배치 크기가 5이기 때문에 5개의 샘플이 출력되는 것을 볼 수 있습니다.

각 샘플의 길이는 20의 길이를 가지는데, 이는 앞서 초기에 필드를 정의할 때 fix_length를 20으로 정해주었기 때문입니다. 

다시 말해 **`하나의 미니 배치의 크기는 (배치 크기 × fix_length)입니다.`**

이제 batch_first를 False로 한 경우를 보겠습니다.

## 4. 필드 재정의하기(torchtext.data)

In [17]:
# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

TEXT 필드에서 batch_first = True 인자를 제거하였습니다. 기본값이 False이므로 batch_first의 값이 False가 됩니다.

## 5. batch_first = False로 하였을 경우의 텐서 크기

In [18]:
# TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행합니다.
train_data, test_data = TabularDataset.splits(path='.', train='train_data.csv', test='test_data.csv', format='csv', fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

# 정의한 필드에 build_vocab() 도구를 사용하면 단어 집합을 생성한다.
TEXT.build_vocab(train_data ,min_freq=10, max_size=10000)

In [19]:
batch_size = 5

train_loader = Iterator(dataset=train_data, batch_size=batch_size)
test_loader = Iterator(dataset=test_data, batch_size=batch_size)

batch = next(iter(train_loader))

In [20]:
print(batch.text)

tensor([[ 610,  451,    9, 1926,    9],
        [ 610,  868,  202,    6,   14],
        [ 610,    2,    9,    2,    0],
        [ 610,    0,   14,  204,   15],
        [ 610,    0,    3,   10,    9],
        [  48,   34,  107,   20,  267],
        [ 408,  276, 9742,   14,    3],
        [  64,   36,   50, 1129,  533],
        [   9,    0,    9,    8,  397],
        [2974, 5154, 6415,    0, 7374],
        [3015,    2,    2,   12,    5],
        [2354,   25,  638,    7,  379],
        [ 202,  315,    6,  595,    0],
        [  26, 8129,  109,  823, 1673],
        [  71,    7,    0,    6,    0],
        [1906,    2, 2249,  180,    0],
        [   6,   59,  706,  123,   34],
        [   0, 1600, 1751,    0,    2],
        [ 185,  991, 2294,    0,   88],
        [  28,   49,    6,  124, 8212]])


In [21]:
print(batch.text.shape)

torch.Size([20, 5])


하나의 미니 배치의 크기는 (fix_length × 배치 크기)입니다.